In [4]:
import pandas as pd
import requests

#From Jason
url_2020 = "http://www.espn.com/nba/attendance"
base_url_other = "http://www.espn.com/nba/attendance/_/year/"


In [5]:
years = [year for year in range(2001,2019)]

for year in years:
    urls.append(base_url_other + str(year))

NameError: name 'urls' is not defined

In [ ]:
nba2001 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2001", header = 0, index_col = 1)
nba2002 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2002", header = 0, index_col = 1)
nba2003 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2003", header = 0, index_col = 1)
nba2004 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2004", header = 0, index_col = 1)
nba2005 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2005", header = 0, index_col = 1)
nba2006 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2006", header = 0, index_col = 1)
nba2007 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2007", header = 0, index_col = 1)
nba2008 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2008", header = 0, index_col = 1)
nba2009 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2009", header = 0, index_col = 1)
nba2010 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2010", header = 0, index_col = 1)
nba2011 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2011", header = 0, index_col = 1)
nba2012 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2012", header = 0, index_col = 1)
nba2013 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2013", header = 0, index_col = 1)
nba2014 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2014", header = 0, index_col = 1)
nba2015 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2015", header = 0, index_col = 1)
nba2016 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2016", header = 0, index_col = 1)
nba2017 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2017", header = 0, index_col = 1)
nba2018 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2018", header = 0, index_col= 1)
nba2019 = pd.read_html("http://www.espn.com/nba/attendance/_/year/2018", header = 0, index_col = 1)
nba2020 = pd.read_html("http://www.espn.com/nba/attendance", header = 0, index_col = 1)

In [ ]:
nba2001_df = pd.DataFrame(nba2001[0])
nba2002_df = pd.DataFrame(nba2002[0])
nba2003_df = pd.DataFrame(nba2003[0])
nba2004_df = pd.DataFrame(nba2004[0])
nba2005_df = pd.DataFrame(nba2005[0])
nba2006_df = pd.DataFrame(nba2006[0])
nba2007_df = pd.DataFrame(nba2007[0])
nba2008_df = pd.DataFrame(nba2008[0])
nba2009_df = pd.DataFrame(nba2009[0])
nba2010_df = pd.DataFrame(nba2010[0])
nba2011_df = pd.DataFrame(nba2011[0])
nba2012_df = pd.DataFrame(nba2012[0])
nba2013_df = pd.DataFrame(nba2013[0])
nba2014_df = pd.DataFrame(nba2014[0])
nba2015_df = pd.DataFrame(nba2015[0])
nba2016_df = pd.DataFrame(nba2016[0])
nba2017_df = pd.DataFrame(nba2017[0])
nba2018_df = pd.DataFrame(nba2018[0])
nba2019_df = pd.DataFrame(nba2019[0])
nba2020_df = pd.DataFrame(nba2020[0])